In [1]:
# need to use updated master JupiterCVML code
import sys
sys.path.append('/home/li.yu/code/JupiterCVML/europa/base/src/europa')

import os
os.environ["BRT_ENV"] = 'prod'
import json
import ndjson
import random
import brtdevkit
print(brtdevkit.__version__)
brtdevkit.log = 'info'
import os
os.environ['AWS_PROFILE'] = 'default'

import time
import numpy as np
import pandas as pd
from datetime import datetime, timezone

from brtdevkit.core.db.athena import AthenaClient, Table
from brtdevkit.data import Image, Dataset

from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
from aletheia_dataset_creator.config.dataset_config import *

pd.set_option('display.max_columns', None)

%matplotlib inline
%load_ext autoreload
%autoreload 2

Internal AWS credentials have been removed from brt-devkit.
To setup AWS credentials, Please follow instructions at : https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html


7.4.3


In [2]:
print(LEFT_CAMERAS)
print(ALL_CAMERA_PAIRS_LIST)

['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']
[{'front-center-left': 'front-center-right', 'front-left-left': 'front-left-right', 'front-right-left': 'front-right-right', 'side-left-left': 'side-left-right', 'side-right-left': 'side-right-right', 'rear-left': 'rear-right', 'front-center-right': 'front-center-left', 'front-left-right': 'front-left-left', 'front-right-right': 'front-right-left', 'side-left-right': 'side-left-left', 'side-right-right': 'side-right-left', 'rear-right': 'rear-left'}, {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}, {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}]


In [3]:
# test_dataset = Dataset.retrieve(id='64ed2657926aefcd654e8269')
test_dataset = Dataset.retrieve(name='dynamic_manny_in_dust_raw')
print(test_dataset.id)
test_df = test_dataset.to_dataframe()
test_df.shape

64fa00ea7b198e7e8de6596d


In [4]:
test_df[['id', 'camera_location']].groupby('camera_location').count()

(151412, 111)

,id
camera_location,
front-center-left,13502
front-center-right,13502
front-left-left,12790
front-left-right,12790
front-right-left,13624
front-right-right,13624
rear-left,11472
rear-right,11472
side-left-left,12181


In [2]:
datasets = [
    '20240119_halo_rgb_left_day_less_night_al_to_label',
    '20240119_halo_rgb_diversity_left_daytime',
    '20231219_halo_rgb_al_to_label',
    '20231219_halo_rgb_diversity_to_label',
    '20231206_halo_rgb_diversity_train_to_label_pt1',
    '20231206_halo_rgb_al_train_to_label_pt1',
    '20231206_halo_rgb_stereo_fps_dedup',
    '20231219_halo_rgb_stereo_fps_dedup',
]
test_dfs = []
for dataset in datasets:
    test_dataset = Dataset.retrieve(name=dataset)
    test_df = test_dataset.to_dataframe()
    print(dataset, test_df.shape)
    test_dfs.append(test_df)
pd.concat(test_dfs, ignore_index=True).to_csv('/data/jupiter/li.yu/data/halo_hard_cases/20231206_20231219_sent_for_labeling.csv', index=False)

20240119_halo_rgb_left_day_less_night_al_to_label (7457, 146)
20240119_halo_rgb_diversity_left_daytime (4128, 146)
20231219_halo_rgb_al_to_label (7552, 147)
20231219_halo_rgb_diversity_to_label (1375, 144)
20231206_halo_rgb_diversity_train_to_label_pt1 (4955, 157)
20231206_halo_rgb_al_train_to_label_pt1 (17049, 157)
20231206_halo_rgb_stereo_fps_dedup (15319, 158)
20231219_halo_rgb_stereo_fps_dedup (5584, 147)


## Run query

In [9]:
# hard_drive_name = 'JUPD-048_2022-09-27'
start_datetime  = datetime(2021, 12, 16, hour=22, minute=14, second=0) #, tzinfo=timezone.utc)
end_datetime  = datetime(2021, 12, 16, hour=22, minute=14, second=35) #, tzinfo=timezone.utc)
# start_datetime  = datetime(2022, 9, 28, tzinfo=timezone.utc) #, hour=3, minute=33, second=33) #, tzinfo=timezone.utc)
# start_datetime  = datetime(2023, 8, 28, hour=0, minute=0)   #, tzinfo=timezone.utc)
# end_datetime    = datetime(2023, 8, 30, hour=23, minute=59)  # this is EXclusive

database = "mesa-data-catalog-prod"
table = "image_jupiter"

# query = \
# f"""
# SELECT *
# FROM {table} T
# WHERE 
#     T.collected_on >= cast('{start_datetime}' as timestamp)
#     AND T.collected_on < cast('{end_datetime}' as timestamp)
#     AND T.robot_name = 'loamy_733'
# """

query = """
SELECT id, image, properties__json FROM annotation_jupiter
WHERE style = 'categorical'
AND created_at > date('2024-01-01')
AND state in ('review', 'ok')
AND properties__json LIKE '%"title": "Pose"%' 
AND kind = 'labelbox'
"""

"""
SELECT
        *
    FROM annotation_jupiter
    WHERE
     annotation_jupiter.image IN ('642607df1620247e2285d6ac')
"""

"""
SELECT id, camera_location, operation_time, collected_on, state, county, operating_field_name, hdr_mode, robot_name, special_notes
T.collected_on >= cast('{start_datetime}' as timestamp)
    AND T.collected_on < cast('{end_datetime}' as timestamp)
    AND T.camera_location LIKE 'front-%'
    AND T.camera_location LIKE '%-left'
    AND T.hard_drive_name = 'JUPD-154_2023-01-14'
    AND T.robot_name LIKE 'loamy%'
    AND T.operation_time != 'daytime'
    AND T.has_nearby_stop_event = true
ORDER BY RAND()
LIMIT 40000
"""

start = time.time()
athena = AthenaClient()
df = athena.get_df(query)
end = time.time()
print(end - start, 's')
df.shape, len(df) // 2

9.540740966796875 s


((122882, 3), 61441)

In [10]:
df.head()

,id,image,properties__json
0,65c1834cbad271dacf97e0e7,5f81f1d7bf0c3674c05778c4,"[{""featureId"": ""cls9tzeqs000a356l9v59fjwt"", ""v..."
1,65c18282ba421ac96fb33c79,60679aa31e3d26cab0c8c694,"[{""featureId"": ""clsa1ufc10006356ns6mt2ud2"", ""v..."
2,65c1827cba421ac96fb338ce,60418162c4cec603cdeb7342,"[{""featureId"": ""clsa1gnjs000h356ngrfqgh2d"", ""v..."
3,65c1825eba421ac96fb32638,5df92c0680a73f861b0c9893,"[{""featureId"": ""clsa1dak10014356nbxbm60n0"", ""v..."
4,65c181517f183b14fcee0d43,6359f3e9d43b44f71072ee9e,"[{""featureId"": ""cls9uwzj5000e356nmlpgj5kv"", ""v..."


In [13]:
df.to_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/all_human_pose.csv', index=False)

In [41]:
id = '61c40f6d645b3a75242fb494'
print(df[df.id == id].shape)
sub_df = df[df.camera_location.str.startswith('rear')]
print(sub_df.shape)
ids = sub_df[sub_df.camera_location == 'rear-left'].sort_values('collected_on').id.to_list()
i = ids.index(id)
print(f'{id} index {i} out of {len(ids)} images')
for j in range(max(0, i-3), min(i+4, len(ids))):
    print(ids[j])

(1, 253)
(142, 253)
61c40f6d645b3a75242fb494 index 51 out of 71 images
61c41049e5100b0f858cc85b
61c40fc732160727091887d5
61c41029ae6321c259440152
61c40f6d645b3a75242fb494
61c40f7e6b0c478c85db4282
61c41059ec85fdd36b953ac2
61c40ff840a09b978c74a5ac


In [45]:
df[['id', 'camera_location']].groupby('camera_location').count()

,id
camera_location,
front-right-left,49
front-right-right,49
rear-left,71
rear-right,71
side-right-left,88
side-right-right,88


In [43]:
# dfs = []
dfs.append(sub_df)
print(len(dfs))

3


In [44]:
df = pd.concat(dfs, ignore_index=True)
df.shape

(416, 253)

## Create a dataset

In [3]:
# df = pd.read_csv('./dust_threshold/human_vehicle_in_dust_for_train_total.csv')
# df = pd.read_csv('./dust_threshold/dust_productivity.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/driveable_not_labeled.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/less_than_half_labeled.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/less_than_half_or_driveable_not_labeled_for_relabeling.csv')
# df = pd.read_csv('/data/jupiter/li.yu/exps/driveable_terrain_model/rgb_baseline_sample_a_v3_2/20230925_halo_rgb_stereo_train_v3_epoch0/pruned_ids.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/Jupiter_train_v6_2/train_v6_2_categorical_count.csv')
df = pd.read_csv('/data/jupiter/li.yu/data/halo_hard_cases/20231219_20240119_halo_rgb_left_airborne.csv')
print(df.shape, 'id' in df, 'unique_id' in df)

(5903, 159) True False


/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (29,56,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
df2 = df[(df.bag_date == '01_08_2024') | (df.bag_date == '01_09_2024')]
df2.shape

(1540, 159)

In [12]:
# unlabeled dataset
Dataset.create(
    name="20240119_halo_rgb_snow",
    description="1540 snow images selected from 20240119_halo_rgb_stereo",
    kind='image',  # annotation or image
    image_ids=df2.id.to_list())

{}

In [26]:
# imageids_to_dataset(
#     image_ids=test_df[test_df.camera_location.str.endswith('left')].id.to_list(),
#     dataset_name="20230823_labeled_right_images_12k_left_images_labels",
#     dataset_description="left images and labels of 20230823_labeled_right_images_12k",
#     dataset_kind='annotation',  # annotation or image
#     mode='mono',  # stereo or mono
#     camera_location=CORE_LEFT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
# )
# imageids_to_dataset(
#     image_ids=test_df[test_df.camera_location.str.endswith('right')].id.to_list(),
#     dataset_name="20230823_labeled_right_images_12k_right_images_labels",
#     dataset_description="right images and labels of 20230823_labeled_right_images_12k",
#     dataset_kind='annotation',  # annotation or image
#     mode='mono',  # stereo or mono
#     camera_location=CORE_RIGHT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
# )

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Filtered 308 annotations.
Sending 12166 annotated_ids for creating dataset
Time taken to prepare data for dataset creation job: 2.07 mins
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Filtered 0 annotations.
Sending 12166 annotated_ids for creating dataset
Time taken to prepare data for dataset creation job: 2.15 mins


In [7]:
# # in master branch halo datasets
# annotations = imageids_to_annotation_df(
#     image_ids=df.id.to_list(),
# )

imageids_to_dataset(
    image_ids=df.id.to_list(),
    dataset_name="20231219_20240119_halo_rgb_airborne",
    dataset_description="labeled images of 20231219_20240119_halo_rgb_left_airborne",
    dataset_kind='annotation',  # annotation or image
    mode='stereo',  # stereo or mono
)
# Dataset.create(
#     name="20230925_halo_rgb_stereo_train_v3_pruned",
#     description="180 images with >0.5 focal loss from dataset 20230925_halo_rgb_stereo_train_v3",
#     kind=Dataset.KIND_ANNOTATION,
#     image_ids=df.id.to_list())

Total annotations retrieved from athena 2978
Removed annotations in state SKIP, 2978 left
Filtered 2978 annotations with non-production label map.
Removed annotations if the label map is not the stanadard production one, 0 left
Removed duplicate annotations for the same image (choosing latest), 0 left
Warning 5903 images do not have a corresponding annotation.


ValueError: No objects to concatenate

## Download a dataset

In [8]:
dataset_name = 'Jupiter_train_v6_2_birds_airborne_debris'
# dataset_dir = os.path.join('/data/jupiter/datasets', dataset_name)
dataset_dir = os.path.join('/data/jupiter/li.yu/data', dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

test_dataset = Dataset.retrieve(name=dataset_name)
test_df = test_dataset.to_dataframe()
test_dataset.download(dataset_dir, df=test_df)

2024-01-23 10:50:10,973 - Dataset - INFO - start downloading dataset: 65affafee23871d5ea8201b2 into /data/jupiter/li.yu/data/Jupiter_train_v6_2_birds_airborne_debris
INFO:Dataset:start downloading dataset: 65affafee23871d5ea8201b2 into /data/jupiter/li.yu/data/Jupiter_train_v6_2_birds_airborne_debris
100%|██████████| 107708/107708 [03:25<00:00, 525.14it/s]
2024-01-23 10:54:29,007 - Dataset - INFO - finished downloading dataset: 65affafee23871d5ea8201b2
INFO:Dataset:finished downloading dataset: 65affafee23871d5ea8201b2
